# EQUITY VANILLA EUROPEAN-STYLE OPTIONS

Valuation and Risk of a simple Vanilla Equity Option

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils.date import Date
from financepy.products.equity import *
from financepy.market.curves import *

####################################################################
#  FINANCEPY BETA Version 0.350 - This build: 30 Apr 2024 at 21:20 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



In the library products are created and then valued. We should be able to vectorise the product inputs and the valuation inputs separately and jointly.

# 1 - Define and value the Call Options using Vectorised Product Inputs

We consider an equity call option. The inputs are:
1) Expiry date
2) Strike price
3) Option Type (call or put)

We start with the strikes as this is the easiest.

I just define some initial non-product inputs

In [3]:
stock_price = 100.0
volatility = 0.20
interest_rate = 0.05
dividend_yield = 0.0

In [4]:
value_dt = Date(20, 1, 2020)

In [5]:
discount_curve = DiscountCurveFlat(value_dt, interest_rate, FrequencyTypes.CONTINUOUS)

In [6]:
dividend_curve = DiscountCurveFlat(value_dt, dividend_yield, FrequencyTypes.CONTINUOUS)

In [7]:
model = BlackScholes(volatility)

## Vectorising the Strikes

In [8]:
expiry_dt = Date(20, 6, 2020)

In [9]:
strike_price = 100.0

In [10]:
call_option = EquityVanillaOption(expiry_dt, strike_price, OptionTypes.EUROPEAN_CALL)

In [11]:
call_option

OBJECT TYPE: EquityVanillaOption
EXPIRY DATE: 20-JUN-2020
STRIKE PRICE: 100.0
OPTION TYPE VALUE: OptionTypes.EUROPEAN_CALL
NUMBER: 1.0

Let's do a valuation

In [12]:
call_option.value(value_dt, stock_price, discount_curve, dividend_curve, model)

6.185679574917835

Now we vectorise the strikes

In [13]:
strike_price = np.linspace(100,150,10)

In [14]:
call_option = EquityVanillaOption(expiry_dt, strike_price, OptionTypes.EUROPEAN_CALL)

In [15]:
call_option

OBJECT TYPE: EquityVanillaOption
EXPIRY DATE: 20-JUN-2020
STRIKE PRICE: [100.         105.55555556 111.11111111 116.66666667 122.22222222
 127.77777778 133.33333333 138.88888889 144.44444444 150.        ]
OPTION TYPE VALUE: OptionTypes.EUROPEAN_CALL
NUMBER: 1.0

Now see if this can be vectorised

In [16]:
call_option.value(value_dt, stock_price, discount_curve, dividend_curve, model)

array([6.18567957, 3.71227186, 2.07068602, 1.07657716, 0.52399448,
       0.23998124, 0.10397414, 0.04284297, 0.01687534, 0.00638438])

Yep this works.

## Vectorising the Expiry Dates FAILS

In [17]:
expiry_dts = value_dt.add_years(np.linspace(0.5,10,20))

In [18]:
expiry_dts

[20-JUL-2020,
 20-JAN-2021,
 20-JUL-2021,
 20-JAN-2022,
 20-JUL-2022,
 20-JAN-2023,
 20-JUL-2023,
 20-JAN-2024,
 20-JUL-2024,
 20-JAN-2025,
 20-JUL-2025,
 20-JAN-2026,
 20-JUL-2026,
 20-JAN-2027,
 20-JUL-2027,
 20-JAN-2028,
 20-JUL-2028,
 20-JAN-2029,
 20-JUL-2029,
 20-JAN-2030]

Make the strike a single number

In [19]:
strike_price = 100.0

In [20]:
call_option = EquityVanillaOption(expiry_dts, strike_price, OptionTypes.EUROPEAN_CALL)

In [21]:
call_option

OBJECT TYPE: EquityVanillaOption
EXPIRY DATE: [20-JUL-2020, 20-JAN-2021, 20-JUL-2021, 20-JAN-2022, 20-JUL-2022, 20-JAN-2023, 20-JUL-2023, 20-JAN-2024, 20-JUL-2024, 20-JAN-2025, 20-JUL-2025, 20-JAN-2026, 20-JUL-2026, 20-JAN-2027, 20-JUL-2027, 20-JAN-2028, 20-JUL-2028, 20-JAN-2029, 20-JUL-2029, 20-JAN-2030]
STRIKE PRICE: 100.0
OPTION TYPE VALUE: OptionTypes.EUROPEAN_CALL
NUMBER: 1.0

In [22]:
call_option.value(value_dt, stock_price, discount_curve, dividend_curve, model)

array([25.76819949, 26.7333646 , 27.69730767, 28.68872885, 29.67439658,
       30.68604223, 31.68973866, 32.71749364, 33.73824534, 34.77625394,
       35.80453806, 36.85365239, 37.8884235 , 38.94217658, 39.97960528,
       41.03385005, 42.07307997, 43.1221029 , 44.15378754, 45.19888358])

This works.

## Vectorising the Enum Type

In [23]:
option_types = list(OptionTypes)[0:2]

In [24]:
option_types

[<OptionTypes.EUROPEAN_CALL: 1>, <OptionTypes.EUROPEAN_PUT: 2>]

In [25]:
call_option = EquityVanillaOption(expiry_dt, strike_price, option_types)

In [26]:
call_option.value(value_dt, stock_price, discount_curve, dividend_curve, model)

array([6.18567957, 4.13058771])

This works but only if the number of elements is the same for all of the inputs.

Copyright (c) 2020 Dominic O'Kane